<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# New LinkedIn followers analysis (creating a simple data pipeline via the notebook in NAAS, a google sheet, and Tableau 
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Google%20Sheets/Google_Sheets_Send_data.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #googlesheets #gsheet #data #naas_drivers #operations #snippet

**Author:** [Asif Syed](https://www.linkedin.com/in/ACoAAAJHE7sB5OxuKHuzguZ9L6lfDHqw--cdnJg/)

Pre-requisite: Google Sheet needs to be shared with NAAS' service account (mentioned below)

🔗 naas-share@naas-gsheets.iam.gserviceaccount.com

Go to the Google Sheet -> File -> Share -> Share with others -> Enter the above service account.

## Input

### Import library

In [1]:
from naas_drivers import gsheet
from naas_drivers import linkedin
import pandas as pd
import numpy as np

### Variables

### Get your cookies
<a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

You can extract the spreadsheet ID from a Google Sheets URL:

Structure of a google sheet url is as follows - https{:}//docs{.}google{.}com/spreadsheets/d/spreadsheetId/edit#gid=0

Replace the strings 'li_at','jession_id' and 'spreadsheet_id' below with the actual data

In [2]:
LI_AT = 'li_at'  # EXAMPLE AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = 'jsession_id'  # EXAMPLE ajax:8379907400220387585

In [2]:
spreadsheet_id = "spreadsheet_id"

## Model

### Connect to LinkedIn

In [3]:
linkedin.connect(LI_AT, JSESSIONID)

### Connect to gsheet

In [4]:
gsheet.connect(spreadsheet_id)

### Get the profiles of last 'n' followers (n is the value assigned to limit parameter below)

In [5]:
df = linkedin.network.get_followers(limit = 50)
profiles = df['PROFILE_URL']

### Get details like Industry name and location for each profile

In [6]:
df_identity = pd.DataFrame()
df1 = pd.DataFrame()
for counter,profile in enumerate(profiles):
    df1 = linkedin.profile.get_identity(profile_url = profile)
    df_identity = df_identity.append(df1)


### Get details like distance and number of followers for each profile

In [7]:
df1 = pd.DataFrame()
df_network = pd.DataFrame()
for counter,profile in enumerate(profiles):
    df1 = linkedin.profile.get_network(profile_url = profile)
    df_network = df_network.append(df1)

### merging three dataframes

In [8]:
df_intermediate = pd.merge(df, df_identity, how = 'inner', on = 'PROFILE_URL')
df_final = pd.merge(df_intermediate, df_network, how = 'inner', on = 'PROFILE_URL')

In [9]:
df_final.drop(columns = ['FIRSTNAME_x','LASTNAME_x','BIRTHDATE','ADDRESS','PROFILE_PICTURE_x','PROFILE_ID_y','SUMMARY'], axis = 1, inplace = True) #dropping personal information and a few redundant columns

## Output

### Send the data

In [10]:
gsheet.send(
    sheet_name="Data",
    data = df_final
)

{'insertedRow': 50}